Working!!

In [2]:
import os
import numpy as np
import pandas as pd
import pathlib
import re
import PyPDF2

In [2]:
def extractPDF(text, ahliDewan, fileName, curPage):
    listDialog = []

    for namaTarget in ahliDewan.Nama:
        text = re.sub(namaTarget, '#namaTarget', text)

    for seat in ahliDewan.Kawasan:
        pattern = '\[' + seat + '.*?\#namaTarget'
        matches = re.findall(pattern, text, flags=re.IGNORECASE)

        for match in matches:
            dialog = {
                'dialog': match,
                'session': fileName,
                'page': curPage             
            }
            listDialog.append(dialog)

    return listDialog

In [3]:
# pathing
PARENT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
DATA_PATH = os.path.join(PARENT_PATH, 'data')
PDF = os.path.join(DATA_PATH, 'PDF')
TEXT = os.path.join(DATA_PATH, 'TEXT')
COMPILE_TEXT = os.path.join(TEXT, 'compile.txt')
DIALOG = os.path.join(TEXT, 'compileDialog.txt')
AHLI_DEWAN2 = os.path.join(DATA_PATH, 'ahliDewan2.csv')

TEST = os.path.join(PDF, 'hansard_14-03-k01-01.pdf')

In [4]:
ahliDewan2 = pd.read_csv(AHLI_DEWAN2)

In [5]:
listDialog = pd.DataFrame()

for head, body, files in os.walk(PDF):
    for file in files:
        with open(TEST, 'rb') as reader:
                # create reader variable that will read the pdffileobj
                pdfreader=PyPDF2.PdfFileReader(reader, strict=False)

                # get total pages
                totalPages = pdfreader.numPages

                for curPages in range(totalPages):
                    pageObj = pdfreader.getPage(curPages)
                    text = pageObj.extractText()
                    text = ' '.join(text.split())
                    
                    # marker
                    text = extractPDF(text, ahliDewan2, file, curPages)

                    # store in df
                    if text:
                        text = pd.DataFrame(text)
                        listDialog = pd.concat([listDialog, text], ignore_index=True)
        

In [6]:
listDialog

,dialog,session,page
0,[Port Dickson]: [Bangun] Tuan Yang di -Pertua:...,hansard_14-03-k01-01.pdf,10
1,"[Semporna]: Tuan Yang di -Pertua, saya... Dato...",hansard_14-03-k01-01.pdf,10
2,[Semporna]: ...Peraturan 11. Tuan Yang di -Per...,hansard_14-03-k01-01.pdf,10
3,"[Bandar Kuching]: Tuan Yang di -Pertua, izinka...",hansard_14-03-k01-01.pdf,10
4,"[Bandar Kuching]: Okey, terima kasih. Tuan Yan...",hansard_14-03-k01-01.pdf,10
...,...,...,...
76001,"[Jelutong]: Tuan Yang di -Pertua, kenap a Tuan...",hansard_14-05-k01-01.pdf,115
76002,"[Jelutong]: No, Tuan Yang di -Pertua. Tuan Yan...",hansard_14-05-k01-01.pdf,115
76003,[Jelutong]: Why are you hijacking the House Tu...,hansard_14-05-k01-01.pdf,115
76004,[Jelutong]: Kami ingin tahu. You are hijacking...,hansard_14-05-k01-01.pdf,116


In [8]:
listDialog.sort_values('page')

,dialog,session,page
0,[Port Dickson]: [Bangun] Tuan Yang di -Pertua:...,hansard_14-03-k01-01.pdf,10
12810,[Port Dickson]: [Bangun] Tuan Yang di -Pertua:...,hansard_14-04-01-12.pdf,10
12811,"[Semporna]: Tuan Yang di -Pertua, saya... Dato...",hansard_14-04-01-12.pdf,10
12812,[Semporna]: ...Peraturan 11. Tuan Yang di -Per...,hansard_14-04-01-12.pdf,10
12813,"[Bandar Kuching]: Tuan Yang di -Pertua, izinka...",hansard_14-04-01-12.pdf,10
...,...,...,...
60633,"[Jelutong]: Tuan Yang di -Pertua, tolonglah be...",hansard_14-05-01-16.pdf,116
15371,"[Jelutong]: Tuan Yang di -Pertua, tolonglah be...",hansard_14-04-01-14.pdf,116
15370,[Jelutong]: Kami ingin tahu. You are hijacking...,hansard_14-04-01-14.pdf,116
35012,[Jelutong]: Kami ingin tahu. You are hijacking...,hansard_14-04-02-20.pdf,116
